In [5]:
!pip install groq


In [7]:
import getpass
from groq import Groq

api_key = getpass.getpass("Enter your GROQ API Key: ")
client = Groq(api_key=api_key)

MODEL_NAME = "llama-3.1-8b-instant"

# =====================================
# DEFINE EXPERT CONFIG
# =====================================

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Senior Technical Support Engineer.
Be precise, rigorous, and code-focused.
Provide debugging steps, code fixes, and clear explanations.
Avoid unnecessary fluff."""
    },
    "billing": {
        "system_prompt": """You are a Customer Billing Support Specialist.
Be empathetic and professional.
Focus on subscription issues, refunds, invoices, and payment disputes.
Reference policy clearly when needed."""
    },
    "general": {
        "system_prompt": """You are a helpful and friendly AI assistant.
Handle general questions and casual conversation professionally."""
    },
    "tool": {
        "system_prompt": "You are a financial data assistant."
    }
}

# =====================================
# ROUTER FUNCTION
# =====================================

def route_prompt(user_input: str) -> str:
    """
    Classifies user input into:
    technical, billing, general, tool
    Returns ONLY the category name.
    """

    routing_prompt = f"""
Classify the following text into one of these categories:
[technical, billing, general, tool]

Return ONLY the category name.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are an intent classifier."},
            {"role": "user", "content": routing_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content.strip().lower()


# =====================================
# MOCK TOOL FUNCTION
# =====================================

def get_bitcoin_price():
    # Mock data (replace with real API later if needed)
    return "The current price of Bitcoin is $63,500 (mock data)."


# =====================================
# ORCHESTRATOR
# =====================================

def process_request(user_input: str) -> str:
    """
    1. Routes the request
    2. Selects expert
    3. Executes tool OR LLM
    4. Returns final response
    """

    category = route_prompt(user_input)

    if category not in MODEL_CONFIG:
        category = "general"

    # 🔹 Tool handling (before LLM call)
    if category == "tool":
        if "bitcoin" in user_input.lower():
            return get_bitcoin_price()

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content


# =====================================
# MAIN LOOP
# =====================================

if __name__ == "__main__":
    print("\n=== Smart Customer Support Router (MoE) ===")

    while True:
        user_query = input("\nEnter your query (type 'exit' to quit): ")

        if user_query.lower() == "exit":
            print("Goodbye 👋")
            break

        try:
            result = process_request(user_query)
            print("\nResponse:\n", result)
        except Exception as e:
            print("\nError:", e)


Enter your GROQ API Key: ··········

=== Smart Customer Support Router (MoE) ===

Enter your query (type 'exit' to quit): How do I fix a NullPointerException in Java?

Response:
 **NullPointerException (NPE) in Java: Causes and Fixes**

A `NullPointerException` occurs when your code attempts to use an object reference that has a null value. Here are the common causes and steps to fix this error:

### Causes:

1. **Null Object Reference**: Your code tries to access a method or property of an object that has not been initialized.
2. **Uninitialized Variables**: Variables are not initialized before they are used.

### Debugging Steps:

1. **Identify the Line of Code**: Determine the exact line where the NPE occurs.
2. **Check Variable Initialization**: Verify that the variable is initialized before it is used.
3. **Check Object References**: Ensure that object references are not null before accessing them.

### Code Fixes:

**1. Initialize Variables**

```java
String name; // Declare the 